In [1]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

In [3]:
# McKay Thompson series from table https://oeis.org/A097340/b097340.txt:
mt=[
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]

In [ ]:
## printed output has been snipped

import pickle


data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w')
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()


# print-output snipped

In [7]:
import pickle
jjfile = open('/Users/barrybrent/10mar21no8.txt','r')
jjs=pickle.load(jjfile)
jjfile.close()
jjs=stripQuotationMarks(jjs)
data=[]
for k in [1..12]:
    n=jjs[k][0]
    jpoly = D(jjs[k][1])
    jnf=numericalfactor(jpoly)
    print (n,jnf,mt[n+1][1])
    

(0, 24, 24)
(1, 276, 276)
(2, 2048, 2048)
(3, 11202, 11202)
(4, 49152, 49152)
(5, 184024, 184024)
(6, 614400, 614400)
(7, 1881471, 1881471)
(8, 5373952, 5373952)
(9, 14478180, 14478180)
(10, 37122048, 37122048)
(11, 91231550, 91231550)


In [18]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
print len(s)

121


In [23]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [0..101]:
    n=s[k][0]
    jpoly = D(jjs[k][1])
    jnf=numericalfactor(jpoly)
    data=data+[jnf-mt[n+1][1]]
print data
    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
f=x^2-4
g=x+2
print f.maxima_methods().divide(g)
print (f+1).maxima_methods().divide(g)

[x - 2, 0]
[x - 2, 1]


In [8]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [0..3]:
    n=s[k][0]
    print "n: ",n
    jpoly = D(s[k][1])
    print jpoly.factor()
    print
    

n:  -1
1

n:  0
(24) * x * (x^2 + 4/3)

n:  1
(276) * x^2 * (x^4 - 8/69*x^2 - 16/23)

n:  2
(2048) * (x - 2) * (x + 2) * x^3 * (x^4 - 8/27*x^2 - 16/27)



In [10]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [0..5]:
    n=s[k][0]
    print "n: ",n
    jpoly = s[k][1]
    if n>1:
        div=jpoly.maxima_methods().divide(x^2-4)
        remainder=div[1]
        data=data+[remainder]
print data
        

n:  -1
n:  0
n:  1
n:  2
n:  3
n:  4
[0, 0, 0]


In [16]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    jpoly = s[k][1]
    if n>1:
        div=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
        remainder=div[1]
        data=data+[remainder]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [2..5]:
    n=s[k][0]
    jpoly = s[k][1]
    print jpoly
    div=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
    div=div[0]
    print div
    if n>1:
        tf=div.polynomial(QQ).is_irreducible()
        print (n,tf)
        print

276*x^6 - 32*x^4 - 192*x^2
276*x^2 + 1072
2048*x^9 - 237568/27*x^7 + 32768/27*x^5 + 131072/27*x^3
2048*x^4 - 16384/27*x^2 - 32768/27
(2, True)

11202*x^12 - 122272*x^10 + 332480*x^8 - 51712*x^6 - 155136*x^4
11202*x^6 - 77464*x^4 + 22624*x^2 + 38784
(3, True)

49152*x^15 - 1072627712/1125*x^13 + 4173856768/675*x^11 - 45736787968/3375*x^9 + 1564475392/675*x^7 + 6257901568/1125*x^5
49152*x^8 - 851443712/1125*x^6 + 10651959296/3375*x^4 - 3128950784/3375*x^2 - 1564475392/1125
(4, True)



In [35]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [2..5]:
    n=s[k][0]
    jpoly = s[k][1]
    div=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
    div=div[0]
    if n>1:
        tf=div.polynomial(QQ).is_irreducible()
        data=data+[tf]
        print (n,tf)
        print
print data

(2, True)

(3, True)

(4, True)

[True, True, True]


In [36]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    jpoly = s[k][1]
    div=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
    div=div[0]
    if n>1:
        tf=div.polynomial(QQ).is_irreducible()
        data=data+[tf]
print data

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [ ]:
# just for reference:
import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() 



In [45]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)

for k in [0..5]:
    print "-------------------------------------------------------------"
    n=s[k][0]
    print "n: ",n
    jpoly = s[k][1]
    Jpoly= t[k][1]
    print jpoly.factor()
    print Jpoly.factor()
    #jdiv=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
    #Jdiv=Jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
    #jdiv=jdiv[0]
    #Jdiv=Jdiv[0]
    #print "n: ",n
    #print jdiv
    #print Jdiv
    #print

-------------------------------------------------------------
n:  -1
1
1
-------------------------------------------------------------
n:  0
8*(3*x^2 + 4)*x
3/8*x^2 + 1/2
-------------------------------------------------------------
n:  1
4*(69*x^4 - 8*x^2 - 48)*x^2
69/1024*x^4 - 1/128*x^2 - 3/64
-------------------------------------------------------------
n:  2
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
1/3456*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)
-------------------------------------------------------------
n:  3
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
1/8388608*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)
-------------------------------------------------------------
n:  4
16384/3375*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)*x^5
1/221184000*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)


In [49]:
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)

for k in [0..5]:
    
    n=s[k][0]
    if n>1:
        print "-------------------------------------------------------------"
        print "n: ",n
        jpoly = s[k][1]
        Jpoly= t[k][1]
        print jpoly.factor()
        print Jpoly.factor()
   
       
        jdiv=jpoly.maxima_methods().divide((x^2-4)*x^(n+1))
        Jdiv=Jpoly.maxima_methods().divide(x^2-4)
        jdiv=jdiv[0]
        Jdiv=Jdiv[0]
        print
        print jdiv
        print Jdiv

-------------------------------------------------------------
n:  2
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
1/3456*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)

2048*x^4 - 16384/27*x^2 - 32768/27
1/128*x^4 - 1/432*x^2 - 1/216
-------------------------------------------------------------
n:  3
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
1/8388608*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)

11202*x^6 - 77464*x^4 + 22624*x^2 + 38784
5601/8388608*x^6 - 9683/2097152*x^4 + 707/524288*x^2 + 303/131072
-------------------------------------------------------------
n:  4
16384/3375*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)*x^5
1/221184000*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)

49152*x^8 - 851443712/1125*x^6 + 10651959296/3375*x^4 - 3128950784/3375*x^2 - 1564475392/1125
3/65536*x^8 - 203/288000*x^6 + 20317/6912000*x^4 - 373/432000*x^2 - 373/288000


In [8]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
for k in [0..10]:
    
    n=s[k][0]
    if n>1:
        print "-------------------------------------------------------------"
        print "n: ",n
        jpoly = s[k][1]
        Jpoly= t[k][1]
        print jpoly.factor()
        print Jpoly.factor()
        quot=jpoly.maxima_methods().divide(Jpoly)
        print quot[0]/x^(n+1)
        data=data+[quot[0]/x^(n+1)]
print data

-------------------------------------------------------------
n:  2
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
1/3456*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)
262144
-------------------------------------------------------------
n:  3
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
1/8388608*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)
16777216
-------------------------------------------------------------
n:  4
16384/3375*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)*x^5
1/221184000*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)
1073741824
-------------------------------------------------------------
n:  5
8/729*(16769187*x^10 - 419821596*x^8 + 3737957344*x^6 - 12223806336*x^4 + 3651604224*x^2 + 4868805632)*(x + 2)*(x - 2)*x^6
1/6262062317568*(16769187*x^10 - 419821596*x^8 + 3737957344*x^6 - 12223806336*x^4 + 3651604224*x^2 + 4868805632)*(x + 2)*(x - 2)
68719476736
-----------------------------

In [17]:
print(2^(6*3),2^(6*4))

(262144, 16777216)


In [18]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
for k in [0..10]:
    
    n=s[k][0]
    if n>1:
        print "-------------------------------------------------------------"
        print "n: ",n
        jpoly = s[k][1]
        Jpoly= t[k][1]
        print jpoly.factor()
        print Jpoly.factor()
        quot=jpoly.maxima_methods().divide(Jpoly)
        print quot[0]/x^(n+1)
        data=data+[quot[0]/x^(n+1)-2^(6*(n+1))]
print data

-------------------------------------------------------------
n:  2
2048/27*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)*x^3
1/3456*(27*x^4 - 8*x^2 - 16)*(x + 2)*(x - 2)
262144
-------------------------------------------------------------
n:  3
2*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)*x^4
1/8388608*(5601*x^6 - 38732*x^4 + 11312*x^2 + 19392)*(x + 2)*(x - 2)
16777216
-------------------------------------------------------------
n:  4
16384/3375*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)*x^5
1/221184000*(10125*x^8 - 155904*x^6 + 650144*x^4 - 190976*x^2 - 286464)*(x + 2)*(x - 2)
1073741824
-------------------------------------------------------------
n:  5
8/729*(16769187*x^10 - 419821596*x^8 + 3737957344*x^6 - 12223806336*x^4 + 3651604224*x^2 + 4868805632)*(x + 2)*(x - 2)*x^6
1/6262062317568*(16769187*x^10 - 419821596*x^8 + 3737957344*x^6 - 12223806336*x^4 + 3651604224*x^2 + 4868805632)*(x + 2)*(x - 2)
68719476736
-----------------------------

In [40]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
for k in [0..len(s)-1]:
    
    n=s[k][0]
    if n>1:
        
        jpoly = s[k][1]
        Jpoly= t[k][1]
      
        quot=jpoly.maxima_methods().divide(Jpoly)
        
        data=data+[quot[0]-x^(n+1)*2^(6*(n+1))]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [43]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
for k in [0..5]:
    
    n=s[k][0]
    if n>1:
        
        jpoly = s[k][1]
        Jpoly= t[k][1]
      
        quot=jpoly.maxima_methods().divide(Jpoly)
        print
        print "n: ",n
        print expand(jpoly-Jpoly*x^(n+1)*2^(6*(n+1)))


n:  2
0

n:  3
0

n:  4
0


In [44]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
for k in [0..5]:
    
    n=s[k][0]
    if n>1:
        
        jpoly = s[k][1]
        Jpoly= t[k][1]
      
        quot=jpoly.maxima_methods().divide(Jpoly)
    
        data=data+ [expand(jpoly-Jpoly*x^(n+1)*2^(6*(n+1)))]
        
print data

[0, 0, 0]


In [45]:
import pickle
jfile = open('/Users/barrybrent/10mar21no8.txt','r')
s=pickle.load(jfile)
jfile.close()
s=stripQuotationMarks(s)

Jfile=open('/Users/barrybrent/run14jun21no14.txt','r')
t=pickle.load(Jfile)
Jfile.close()
t=stripQuotationMarks(t)
data=[]
ln=min(len(s),len(t))
for k in [0..ln-1]:
    
    n=s[k][0]
    if n>1:
        
        jpoly = s[k][1]
        Jpoly= t[k][1]
      
        quot=jpoly.maxima_methods().divide(Jpoly)
    
        data=data+ [expand(jpoly-Jpoly*x^(n+1)*2^(6*(n+1)))]
        
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
